In [6]:
!pip install pymongo dnspython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 23.3 MB/s eta 0:00:00


In [7]:
import pymongo
import getpass
from IPython.display import clear_output

In [16]:
def connect_to_mongodb():
  connection_string=getpass.getpass("Connection string: ")

  client=pymongo.MongoClient(connection_string)
  db=client['chatbotDB']
  collection=db['items']

  collection.create_index(
      [("name", "text"),("description","text")],
      name="TextIndex"
  )

  return collection

In [11]:
def search_items(collection,query):
  try:
    results=collection.find(
        {"$text": {"$search":query}},
        {"score":{"$meta": "textScore"}}
    ).sort([("score",{"$meta":"textScore"})]).limit(5)


    result_list=list(results)
    if not result_list:
      return "no matching items found ."

    response="\n".join([f"{item['name']}->{item['description']}" for item in result_list])
    return response
  except Exception as e:
    return f"Error:{str(e)}"

In [12]:
def run_chatbot():
  collection=connect_to_mongodb()
  print("Chatbot started! Type 'exit' to quit.\n")

  while True:
    query =input("Enter your Keyword: ").strip()
    if query.lower() == 'exit':
      print("Chatbot stopped.")
      break
    if not query:
      print("Please print a query.")
      continue

    clear_output(wait=True)
    print(f" You:{query}")
    response=search_items(collection, query)
    print(f"\n  Bot:\n{response}")

In [ ]:
run_chatbot()

Connection string: ··········
Chatbot started! Type 'exit' to quit.

